### TODO: import  this library to make connection to  postgres database

In [5]:
import psycopg2

#!pip install psycopg2-binary
# run !pip install pyscopg2 #if your environment does not have this library

### TODO: Create a connection to the `rocket spark` database, get a cursor, and set autocommit to true

In [19]:
try: 
    conn = psycopg2.connect("host=54.175.254.25 dbname=rocket_spark user=data_analyst password=secret")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get cursor to the Database")
    print(e)
conn.set_session(autocommit=True)

### TODO : Run the cells below to confirm  if all tables exist and how many rows are available for each table

In [20]:
print("\nTable: songs\n")
try: 
    cur.execute("SELECT count(*) FROM songs;")
except psycopg2.Error as e: 
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()


Table: songs

(71,)


In [21]:
print("\nTable: songsplays\n")
try: 
    cur.execute("SELECT count(*) FROM songplays;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()


Table: songsplays

(1368,)


In [22]:
print("\n:Table artists\n")
try: 
    cur.execute("SELECT count(*) FROM artists;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()


:Table artists

(69,)


In [23]:
print("\nTable: users\n")
try: 
    cur.execute("SELECT count(*) FROM users;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()


Table: users

(90,)


In [24]:
print("\nTable: time\n")
try: 
    cur.execute("SELECT count(*) FROM time;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)


Table: time



In [25]:
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()    

(4978,)


> #### Question 1: Find all `users` (first_name, last_name, gender and level) (song title) who have listened to the song  `Face the Ashes` in year 2018

In [60]:
cur.execute("""SELECT users.first_name, 
                users.last_name, 
                users.gender, 
                users.level, 
                songs.title 
                FROM users JOIN songplays JOIN songs
                ON users.user_id = songs.user_id 
                AND songs.song_id = songplays.song_id
                WHERE songs.title = 'Face the Ashes' 
                AND songs.year = 2018; """)


SyntaxError: syntax error at or near "WHERE"
LINE 9:                 WHERE songs.title = 'Face the Ashes' 
                        ^


> #### Question 2:  Provide  top 100 most played songs  `songplays` (song title , location, artist name, duration, (column count for how many times it was played )

In [55]:
cur.execute("""SELECT songs.title, songplays.location, artists.name, songs.duration, count(songplays.song_play_id) AS Times_played
                FROM songplays JOIN songs JOIN artists
                ON songs.song_id = songplays.song_id 
                AND songs.artist_id = artists.artist_id  
                ORDER BY Times_played DESC
                LIMIT 100;""")

SyntaxError: syntax error at or near "ORDER"
LINE 5:                 ORDER BY Times_played DESC
                        ^


> #### Question 3: By explorative data analysis find if there are gainful insights between `location` and `artists` and `song`  played

In [72]:
data = cur.execute (""""SELECT artists.location, artists.name, songs.title, count(songs.title) AS song_titles, 
                count(artists.name) AS names
                FROM songs JOIN artists
                ON artists.artist_id = songs.artist_id
                GROUP BY artists.location;""")


data_df = pd.read_sql(data, con)
#bar plot showing song titles per location
data_df.plot.bar(x=artists.location, y=song_titles)

#bar plot showing count of artist names per location
data_df.plot.bar(x=artists.location, y=names)

SyntaxError: unterminated quoted identifier at or near ""SELECT artists.location, artists.name, songs.title
                FROM songs JOIN artists
                ON artists.artist_id = songs.artist_id;"
LINE 1: "SELECT artists.location, artists.name, songs.title
        ^


> #### Question 4: Using the `song_plays` table, write an SQL query to find the record which is the fourth most listened song.

In [59]:
cur.execute("""SELECT songs.title, count(songplays.song_play_id) AS Times_played
                FROM songplays JOIN songs
                ON songs.song_id = songplays.song_id
                GROUP BY songs.title
                ORDER BY Times_played DESC
                LIMIT 4;""")

> #### Question 5: Based on your analysis derive to what extent is there correlation between `songplays` and `users`level(free , paid )

In [62]:
cur.execute("""SELECT songplays.level, users.level
                FROM songplays, users
                corr(songplays.level, users.level);""")

SyntaxError: syntax error at or near "."
LINE 3:                 corr(songplays.level, users.level);
                                      ^


### Close Connection 


In [73]:
cur.close()
conn.close()